In [1]:
import pandas as pd
import numpy as np


In [2]:
# read tab delimited data
data = pd.read_csv("./data/data.txt", sep="\t")
# remove all na rows
data = data.dropna(axis=0, how="all")
# remove all na columns
data = data.dropna(axis=1, how="all")
# strip strings
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# data.columns.values
data


,SUBJECTIVE SYMPTOM,ASSOCIATED SYMPTOMS,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,INVESTIGATIONS DONE,Unnamed: 14,...,Unnamed: 21,Unnamed: 22,PROVISIONAL DIAGNOSIS,Unnamed: 25,Unnamed: 26,ADVISED INVESTIGATIONS,Unnamed: 32,Unnamed: 33,MANAGEMENT,SURGICAL MANAGEMENT
0,nasal,sneezing,asthma,stress,enhancing factors,itchy throat,red eyes,NaN,blood tests,blood IgE,...,NaN,NaN,allergy,NaN,NaN,antihistamine,nasal spray - steroid,NaN,NaN,NaN
1,nasal,sneezing,allergy,enhancing factors,reducing factors,NaN,NaN,NaN,blood tests,blood IgE,...,NaN,NaN,allergy,NaN,NaN,antihistamine,nasal spray - decongestant,NaN,NaN,NaN
2,nasal,runny nose,skin allergy,anxiety,reducing factors,treatment history,NaN,NaN,blood tests,blood IgE,...,NaN,NaN,allergy,vasomotor,NaN,avoid stress/anxiety,nasal spray - decongestant,NaN,medical then surgical,surgery
3,nasal,sneezing,runny nose,blockage in nose,NaN,NaN,NaN,NaN,blood tests,blood IgE,...,NaN,NaN,allergy,nasal polyposis,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS
4,nasal,runny nose,blockage in nose,NaN,NaN,NaN,NaN,NaN,blood tests,NaN,...,NaN,NaN,vasomotor,deviated nasal septum,nasal polyposis,avoid stress/anxiety,nasal spray - decongestant,NaN,medical and surgical,surgery
5,nasal,blockage in both sides,persistent,NaN,NaN,NaN,NaN,NaN,CT scan PNS,NaN,...,NaN,NaN,nasal polyposis,mass,NaN,antihistamine,nasal spray - decongestant,NaN,medical then surgical,FESS
6,nasal,blockage in one side,persistent,NaN,NaN,NaN,NaN,NaN,CT scan PNS,NaN,...,NaN,NaN,deviated nasal septum,nasal polyposis,NaN,antihistamine,nasal spray - decongestant,NaN,medical and surgical,septal correction
7,nasal,bleeding,stress,anxiety,NaN,NaN,NaN,NaN,bleeding profile,NaN,...,NaN,NaN,nose pricking,hypertension,cancer,avoid stress/anxiety,control causative factors,NaN,NaN,NaN
8,nasal,unable to smell,blockage in nose,discharge,NaN,NaN,NaN,NaN,CT scan PNS,NaN,...,NaN,NaN,nasal polyposis,cancer,trauma,nasal spray - decongestant,NaN,NaN,medical then surgical,surgery
9,nasal,unable to smell,intermittent,NaN,NaN,NaN,NaN,NaN,blood IgE,NaN,...,NaN,NaN,nasal polyposis,allergy,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS


In [3]:
# replace unnamed columns with last named column in lower case and space replaced with _
columns = [
    last_named_column
    if "Unnamed" in column
    else (last_named_column := column.lower().replace(" ", "_"))
    for column in data.columns
]
# # alternatively,
# columns = []
# for column in data.columns:
#     if not "Unnamed" in column:
#         last_named_column = column.lower().replace(" ", "_")
#     columns.append(last_named_column)
data.columns = columns

# # make it all lower case
# data = data.applymap(lambda s: s.lower() if type(s) == str else s)

data

,subjective_symptom,associated_symptoms,associated_symptoms,associated_symptoms,associated_symptoms,associated_symptoms,associated_symptoms,associated_symptoms,investigations_done,investigations_done,...,age,age,provisional_diagnosis,provisional_diagnosis,provisional_diagnosis,advised_investigations,advised_investigations,advised_investigations,management,surgical_management
0,nasal,sneezing,asthma,stress,enhancing factors,itchy throat,red eyes,NaN,blood tests,blood IgE,...,NaN,NaN,allergy,NaN,NaN,antihistamine,nasal spray - steroid,NaN,NaN,NaN
1,nasal,sneezing,allergy,enhancing factors,reducing factors,NaN,NaN,NaN,blood tests,blood IgE,...,NaN,NaN,allergy,NaN,NaN,antihistamine,nasal spray - decongestant,NaN,NaN,NaN
2,nasal,runny nose,skin allergy,anxiety,reducing factors,treatment history,NaN,NaN,blood tests,blood IgE,...,NaN,NaN,allergy,vasomotor,NaN,avoid stress/anxiety,nasal spray - decongestant,NaN,medical then surgical,surgery
3,nasal,sneezing,runny nose,blockage in nose,NaN,NaN,NaN,NaN,blood tests,blood IgE,...,NaN,NaN,allergy,nasal polyposis,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS
4,nasal,runny nose,blockage in nose,NaN,NaN,NaN,NaN,NaN,blood tests,NaN,...,NaN,NaN,vasomotor,deviated nasal septum,nasal polyposis,avoid stress/anxiety,nasal spray - decongestant,NaN,medical and surgical,surgery
5,nasal,blockage in both sides,persistent,NaN,NaN,NaN,NaN,NaN,CT scan PNS,NaN,...,NaN,NaN,nasal polyposis,mass,NaN,antihistamine,nasal spray - decongestant,NaN,medical then surgical,FESS
6,nasal,blockage in one side,persistent,NaN,NaN,NaN,NaN,NaN,CT scan PNS,NaN,...,NaN,NaN,deviated nasal septum,nasal polyposis,NaN,antihistamine,nasal spray - decongestant,NaN,medical and surgical,septal correction
7,nasal,bleeding,stress,anxiety,NaN,NaN,NaN,NaN,bleeding profile,NaN,...,NaN,NaN,nose pricking,hypertension,cancer,avoid stress/anxiety,control causative factors,NaN,NaN,NaN
8,nasal,unable to smell,blockage in nose,discharge,NaN,NaN,NaN,NaN,CT scan PNS,NaN,...,NaN,NaN,nasal polyposis,cancer,trauma,nasal spray - decongestant,NaN,NaN,medical then surgical,surgery
9,nasal,unable to smell,intermittent,NaN,NaN,NaN,NaN,NaN,blood IgE,NaN,...,NaN,NaN,nasal polyposis,allergy,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS


In [4]:
if isinstance(data.associated_symptoms, pd.DataFrame):
    # merge and join all associated_symptoms with "|"
    new_data = pd.DataFrame()
    for _, row in data.iterrows():
        new_data = pd.concat(
            [
                new_data,
                pd.DataFrame(
                    {
                        "associated_symptoms": [
                            "|".join(
                                row.associated_symptoms.dropna().to_list())
                        ]
                    }
                ),
            ]
        )
    data = data.drop(columns="associated_symptoms")
    data.insert(
        1, column="associated_symptoms", value=new_data.associated_symptoms.to_list()
    )

data


,subjective_symptom,associated_symptoms,investigations_done,investigations_done,gender,age,age,age,age,provisional_diagnosis,provisional_diagnosis,provisional_diagnosis,advised_investigations,advised_investigations,advised_investigations,management,surgical_management
0,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests,blood IgE,both,all ages,NaN,NaN,NaN,allergy,NaN,NaN,antihistamine,nasal spray - steroid,NaN,NaN,NaN
1,nasal,sneezing|allergy|enhancing factors|reducing fa...,blood tests,blood IgE,both,all ages,NaN,NaN,NaN,allergy,NaN,NaN,antihistamine,nasal spray - decongestant,NaN,NaN,NaN
2,nasal,runny nose|skin allergy|anxiety|reducing facto...,blood tests,blood IgE,both,all ages,NaN,NaN,NaN,allergy,vasomotor,NaN,avoid stress/anxiety,nasal spray - decongestant,NaN,medical then surgical,surgery
3,nasal,sneezing|runny nose|blockage in nose,blood tests,blood IgE,both,all ages,NaN,NaN,NaN,allergy,nasal polyposis,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS
4,nasal,runny nose|blockage in nose,blood tests,NaN,both,all ages,NaN,NaN,NaN,vasomotor,deviated nasal septum,nasal polyposis,avoid stress/anxiety,nasal spray - decongestant,NaN,medical and surgical,surgery
5,nasal,blockage in both sides|persistent,CT scan PNS,NaN,both,all ages,NaN,NaN,NaN,nasal polyposis,mass,NaN,antihistamine,nasal spray - decongestant,NaN,medical then surgical,FESS
6,nasal,blockage in one side|persistent,CT scan PNS,NaN,both,all ages,NaN,NaN,NaN,deviated nasal septum,nasal polyposis,NaN,antihistamine,nasal spray - decongestant,NaN,medical and surgical,septal correction
7,nasal,bleeding|stress|anxiety,bleeding profile,NaN,both,all ages,NaN,NaN,NaN,nose pricking,hypertension,cancer,avoid stress/anxiety,control causative factors,NaN,NaN,NaN
8,nasal,unable to smell|blockage in nose|discharge,CT scan PNS,NaN,both,all ages,NaN,NaN,NaN,nasal polyposis,cancer,trauma,nasal spray - decongestant,NaN,NaN,medical then surgical,surgery
9,nasal,unable to smell|intermittent,blood IgE,NaN,both,all ages,NaN,NaN,NaN,nasal polyposis,allergy,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS


In [5]:
if isinstance(data.investigations_done, pd.DataFrame):
    # merge and join all investigations_done with "|"
    new_data = pd.DataFrame()
    for _, row in data.iterrows():
        new_data = pd.concat(
            [
                new_data,
                pd.DataFrame(
                    {
                        "investigations_done": [
                            "|".join(
                                row.investigations_done.dropna().to_list())
                        ]
                    }
                ),
            ]
        )
    data = data.drop(columns="investigations_done")
    data.insert(
        2, column="investigations_done", value=new_data.investigations_done.to_list()
    )

data


,subjective_symptom,associated_symptoms,investigations_done,gender,age,age,age,age,provisional_diagnosis,provisional_diagnosis,provisional_diagnosis,advised_investigations,advised_investigations,advised_investigations,management,surgical_management
0,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,both,all ages,NaN,NaN,NaN,allergy,NaN,NaN,antihistamine,nasal spray - steroid,NaN,NaN,NaN
1,nasal,sneezing|allergy|enhancing factors|reducing fa...,blood tests|blood IgE,both,all ages,NaN,NaN,NaN,allergy,NaN,NaN,antihistamine,nasal spray - decongestant,NaN,NaN,NaN
2,nasal,runny nose|skin allergy|anxiety|reducing facto...,blood tests|blood IgE,both,all ages,NaN,NaN,NaN,allergy,vasomotor,NaN,avoid stress/anxiety,nasal spray - decongestant,NaN,medical then surgical,surgery
3,nasal,sneezing|runny nose|blockage in nose,blood tests|blood IgE,both,all ages,NaN,NaN,NaN,allergy,nasal polyposis,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS
4,nasal,runny nose|blockage in nose,blood tests,both,all ages,NaN,NaN,NaN,vasomotor,deviated nasal septum,nasal polyposis,avoid stress/anxiety,nasal spray - decongestant,NaN,medical and surgical,surgery
5,nasal,blockage in both sides|persistent,CT scan PNS,both,all ages,NaN,NaN,NaN,nasal polyposis,mass,NaN,antihistamine,nasal spray - decongestant,NaN,medical then surgical,FESS
6,nasal,blockage in one side|persistent,CT scan PNS,both,all ages,NaN,NaN,NaN,deviated nasal septum,nasal polyposis,NaN,antihistamine,nasal spray - decongestant,NaN,medical and surgical,septal correction
7,nasal,bleeding|stress|anxiety,bleeding profile,both,all ages,NaN,NaN,NaN,nose pricking,hypertension,cancer,avoid stress/anxiety,control causative factors,NaN,NaN,NaN
8,nasal,unable to smell|blockage in nose|discharge,CT scan PNS,both,all ages,NaN,NaN,NaN,nasal polyposis,cancer,trauma,nasal spray - decongestant,NaN,NaN,medical then surgical,surgery
9,nasal,unable to smell|intermittent,blood IgE,both,all ages,NaN,NaN,NaN,nasal polyposis,allergy,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS


In [6]:
if isinstance(data.age, pd.DataFrame):
    # split multiple age values to multiple records
    processed_data = pd.DataFrame()
    for _, row in data.iterrows():
        items = row.age.dropna().to_list()
        new_data = row.copy().to_frame().T
        new_data = new_data.drop(columns="age")
        new_data.insert(
            4,
            column="age",
            value=items[0],
        )
        for item in items[1:]:
            new_data = pd.concat(
                [new_data, new_data.tail(1)], ignore_index=True)
            new_data.at[new_data.shape[0] - 1, "age"] = item
        processed_data = pd.concat(
            [processed_data, new_data], ignore_index=True)

    data = processed_data

data


,subjective_symptom,associated_symptoms,investigations_done,gender,age,provisional_diagnosis,provisional_diagnosis,provisional_diagnosis,advised_investigations,advised_investigations,advised_investigations,management,surgical_management
0,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,both,all ages,allergy,NaN,NaN,antihistamine,nasal spray - steroid,NaN,NaN,NaN
1,nasal,sneezing|allergy|enhancing factors|reducing fa...,blood tests|blood IgE,both,all ages,allergy,NaN,NaN,antihistamine,nasal spray - decongestant,NaN,NaN,NaN
2,nasal,runny nose|skin allergy|anxiety|reducing facto...,blood tests|blood IgE,both,all ages,allergy,vasomotor,NaN,avoid stress/anxiety,nasal spray - decongestant,NaN,medical then surgical,surgery
3,nasal,sneezing|runny nose|blockage in nose,blood tests|blood IgE,both,all ages,allergy,nasal polyposis,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS
4,nasal,runny nose|blockage in nose,blood tests,both,all ages,vasomotor,deviated nasal septum,nasal polyposis,avoid stress/anxiety,nasal spray - decongestant,NaN,medical and surgical,surgery
5,nasal,blockage in both sides|persistent,CT scan PNS,both,all ages,nasal polyposis,mass,NaN,antihistamine,nasal spray - decongestant,NaN,medical then surgical,FESS
6,nasal,blockage in one side|persistent,CT scan PNS,both,all ages,deviated nasal septum,nasal polyposis,NaN,antihistamine,nasal spray - decongestant,NaN,medical and surgical,septal correction
7,nasal,bleeding|stress|anxiety,bleeding profile,both,all ages,nose pricking,hypertension,cancer,avoid stress/anxiety,control causative factors,NaN,NaN,NaN
8,nasal,unable to smell|blockage in nose|discharge,CT scan PNS,both,all ages,nasal polyposis,cancer,trauma,nasal spray - decongestant,NaN,NaN,medical then surgical,surgery
9,nasal,unable to smell|intermittent,blood IgE,both,all ages,nasal polyposis,allergy,NaN,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS


In [7]:
if isinstance(data.provisional_diagnosis, pd.DataFrame):
    # split multiple provisional_diagnosis values to multiple records
    processed_data = pd.DataFrame()
    for _, row in data.iterrows():
        items = row.provisional_diagnosis.dropna().to_list()
        new_data = row.copy().to_frame().T
        new_data = new_data.drop(columns="provisional_diagnosis")
        new_data.insert(
            5,
            column="provisional_diagnosis",
            value=items[0],
        )
        for item in items[1:]:
            new_data = pd.concat(
                [new_data, new_data.tail(1)], ignore_index=True)
            new_data.at[new_data.shape[0] - 1, "provisional_diagnosis"] = item
        processed_data = pd.concat(
            [processed_data, new_data], ignore_index=True)

    data = processed_data

data


,subjective_symptom,associated_symptoms,investigations_done,gender,age,provisional_diagnosis,advised_investigations,advised_investigations,advised_investigations,management,surgical_management
0,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,both,all ages,allergy,antihistamine,nasal spray - steroid,NaN,NaN,NaN
1,nasal,sneezing|allergy|enhancing factors|reducing fa...,blood tests|blood IgE,both,all ages,allergy,antihistamine,nasal spray - decongestant,NaN,NaN,NaN
2,nasal,runny nose|skin allergy|anxiety|reducing facto...,blood tests|blood IgE,both,all ages,allergy,avoid stress/anxiety,nasal spray - decongestant,NaN,medical then surgical,surgery
3,nasal,runny nose|skin allergy|anxiety|reducing facto...,blood tests|blood IgE,both,all ages,vasomotor,avoid stress/anxiety,nasal spray - decongestant,NaN,medical then surgical,surgery
4,nasal,sneezing|runny nose|blockage in nose,blood tests|blood IgE,both,all ages,allergy,antihistamine,nasal spray - steroid,NaN,medical then surgical,FESS
...,...,...,...,...,...,...,...,...,...,...,...
72,abdomen,chronic|pain in lower abdomen|pain during inte...,ultrasound,female,reproductive age,PID-pelvic inflammatory disease,additional investigations,NaN,NaN,gynaecologist opinion,NaN
73,abdomen,severe|pain in lower abdomen|pain on movement|...,ultrasound,female,reproductive age,twisted ovarian cyst,NaN,NaN,NaN,gynaecologist opinion,surgery
74,abdomen,pain in abdomen|nausea|vomiting|local tenderne...,,both,1-12 years,abdominal lymph nodes,additional investigations,NaN,NaN,paediatric surgeon opinion,NaN
75,abdomen,severe|pain in sides of abdomen|vomiting|burni...,ultrasound,both,all ages,ureteric stone,additional investigations,CT IVP - urogram,NaN,urologist opinion,NaN


In [8]:
# expand 'both' gender value to female and male rows
processed_data = pd.DataFrame()
for _, row in data.iterrows():
    if "both" in row.gender:
        new_data = pd.DataFrame()
        for item in ["female", "male"]:
            new_row = row.copy()
            new_row.gender = item
            new_data = pd.concat(
                [new_data, new_row.to_frame().T], ignore_index=True)
    else:
        new_data = row.copy().to_frame().T
    processed_data = pd.concat([processed_data, new_data], ignore_index=True)

data = processed_data

data


,subjective_symptom,associated_symptoms,investigations_done,gender,age,provisional_diagnosis,advised_investigations,advised_investigations,advised_investigations,management,surgical_management
0,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,female,all ages,allergy,antihistamine,nasal spray - steroid,NaN,NaN,NaN
1,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,male,all ages,allergy,antihistamine,nasal spray - steroid,NaN,NaN,NaN
2,nasal,sneezing|allergy|enhancing factors|reducing fa...,blood tests|blood IgE,female,all ages,allergy,antihistamine,nasal spray - decongestant,NaN,NaN,NaN
3,nasal,sneezing|allergy|enhancing factors|reducing fa...,blood tests|blood IgE,male,all ages,allergy,antihistamine,nasal spray - decongestant,NaN,NaN,NaN
4,nasal,runny nose|skin allergy|anxiety|reducing facto...,blood tests|blood IgE,female,all ages,allergy,avoid stress/anxiety,nasal spray - decongestant,NaN,medical then surgical,surgery
...,...,...,...,...,...,...,...,...,...,...,...
134,abdomen,pain in abdomen|nausea|vomiting|local tenderne...,,male,1-12 years,abdominal lymph nodes,additional investigations,NaN,NaN,paediatric surgeon opinion,NaN
135,abdomen,severe|pain in sides of abdomen|vomiting|burni...,ultrasound,female,all ages,ureteric stone,additional investigations,CT IVP - urogram,NaN,urologist opinion,NaN
136,abdomen,severe|pain in sides of abdomen|vomiting|burni...,ultrasound,male,all ages,ureteric stone,additional investigations,CT IVP - urogram,NaN,urologist opinion,NaN
137,abdomen,loose stools|certain foods/beverages|bloating|...,,female,all ages,intolerance to foods/beverages,stool test,CT scan,colonoscopy,stop instigating/enhancing factors,NaN


In [9]:
# expand 'all ages' age value to multiple age group rows
processed_data = pd.DataFrame()
for _, row in data.iterrows():
    if "all ages" in row.age:
        new_data = pd.DataFrame()
        for item in [
            "upto 4 weeks",
            "1-12 months",
            "1-12 years",
            "13-18 years",
            "19-25 years",
            "26-35 years",
            "36-50 years",
            "51-65 years",
            "above 65 years",
        ]:
            new_row = row.copy()
            new_row.age = item
            new_data = pd.concat(
                [new_data, new_row.to_frame().T], ignore_index=True)
    else:
        new_data = row.copy().to_frame().T
    processed_data = pd.concat([processed_data, new_data], ignore_index=True)

data = processed_data

data


,subjective_symptom,associated_symptoms,investigations_done,gender,age,provisional_diagnosis,advised_investigations,advised_investigations,advised_investigations,management,surgical_management
0,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,female,upto 4 weeks,allergy,antihistamine,nasal spray - steroid,NaN,NaN,NaN
1,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,female,1-12 months,allergy,antihistamine,nasal spray - steroid,NaN,NaN,NaN
2,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,female,1-12 years,allergy,antihistamine,nasal spray - steroid,NaN,NaN,NaN
3,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,female,13-18 years,allergy,antihistamine,nasal spray - steroid,NaN,NaN,NaN
4,nasal,sneezing|asthma|stress|enhancing factors|itchy...,blood tests|blood IgE,female,19-25 years,allergy,antihistamine,nasal spray - steroid,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1182,abdomen,loose stools|certain foods/beverages|bloating|...,,male,19-25 years,intolerance to foods/beverages,stool test,CT scan,colonoscopy,stop instigating/enhancing factors,NaN
1183,abdomen,loose stools|certain foods/beverages|bloating|...,,male,26-35 years,intolerance to foods/beverages,stool test,CT scan,colonoscopy,stop instigating/enhancing factors,NaN
1184,abdomen,loose stools|certain foods/beverages|bloating|...,,male,36-50 years,intolerance to foods/beverages,stool test,CT scan,colonoscopy,stop instigating/enhancing factors,NaN
1185,abdomen,loose stools|certain foods/beverages|bloating|...,,male,51-65 years,intolerance to foods/beverages,stool test,CT scan,colonoscopy,stop instigating/enhancing factors,NaN


In [10]:
data.to_csv("./data/data_processed.txt", sep="\t")